Artifical Neural Network

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.rc('axes', labelsize=14)
plt.rc('font', size=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=12)
plt.rc('ytick', labelsize=12)

The Preceptron

In [3]:
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

iris = load_iris(as_frame=True)
X = iris.data[["petal length (cm)", "petal width (cm)"]].values
y = (iris.target==0)

per_clf = Perceptron(random_state=42)
per_clf.fit(X,y)

,penalty,None
,alpha,0.0001
,l1_ratio,0.15
,fit_intercept,True
,max_iter,1000
,tol,0.001
,shuffle,True
,verbose,0
,eta0,1.0
,n_jobs,None
,random_state,42


In [4]:
X_new = [[2, 0.5], [3, 1]]
y_pred = per_clf.predict(X_new) 

In [5]:
y_pred

array([ True, False])

The Perceptron is equivalent to a SGDClassifier with loss="perceptron", no regularization, and a constant learning rate equal to 1:

In [6]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(loss="perceptron",penalty=None,learning_rate="constant",eta0=1,random_state=42)

sgd_clf.fit(X, y)

,loss,'perceptron'
,penalty,None
,alpha,0.0001
,l1_ratio,0.15
,fit_intercept,True
,max_iter,1000
,tol,0.001
,shuffle,True
,verbose,0
,epsilon,0.1
,n_jobs,None


In [7]:
X_new = [[2, 0.5], [3, 1]]
y_pred = sgd_clf.predict(X_new) 

In [8]:
y_pred

array([ True, False])

In [9]:
assert (sgd_clf.coef_ == per_clf.coef_).all()
assert (sgd_clf.intercept_ == per_clf.intercept_).all()

Multilayer Perceptron Regression

In [12]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [14]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split( 
housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split( 
X_train_full, y_train_full, random_state=42) 
mlp_reg = MLPRegressor(hidden_layer_sizes=[50, 50, 50], 
random_state=42)
pipeline = make_pipeline(StandardScaler(), mlp_reg)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_valid)
rmse = root_mean_squared_error(y_valid, y_pred)

In [15]:
rmse

0.5053326657968437

Implement the MLPs with Keras

Image Classifier Using Sequential API

In [ ]:
import tensorflow as tf

fashion_minst = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_minst
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]

In [ ]:
X_train.shape

In [ ]:
X_train.dtype

Scale the values

In [ ]:
X_train, X_valid, X_test = X_train / 255., X_valid / 255., X_test/255.

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress",
"Coat","Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [ ]:
class_names[y_train[0]]

Sequential Model

Method 1: Making model

In [ ]:
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(shape=[28,28]))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(300, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

Method 2: Making model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.Flatten(input_shape=[28,28]),
    tf.keras.Dense(300, activation='relu'),
    tf.keras.Dense(100, activation='relu'),
    tf.keras.Dense(10, activation='softmax'),
])

In [ ]:
model.summary()

In [ ]:
model.layers

In [ ]:
hidden1 = model.layers[1]
hidden1.name

In [ ]:
model.get_layer('dense') is hidden1

In [ ]:
weights, biases = hidden1.get_weights()
weights

In [ ]:
weights.shape

In [ ]:
biases

In [ ]:
biases.shape

In [ ]:
model.compile(
    loss = 'sparse_categorical_crossentropy',
    optimizer='sgd',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

Making the Predictions

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

Getting the highest probability class

In [ ]:
y_pred = y_proba.argmax(axis=-1)
y_pred

In [ ]:
np.array(class_names)[y_pred]

In [ ]:
y_new = y_test[:3]
y_new

Building Regression MLPs using Sequential API